# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [130]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()


In [5]:
print(data)

{'avgTotalVolume': 68302078, 'calculationPrice': 'close', 'change': 2.57, 'changePercent': 0.01657, 'close': 0, 'closeSource': 'aifofcil', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 157.9, 'iexCloseTime': 1679482758463, 'iexLastUpdated': 1640659126382, 'iexMarketPercent': 0.00038299629995865224, 'iexOpen': 154.4, 'iexOpenTime': 1701068439093, 'iexRealtimePrice': 154.23, 'iexRealtimeSize': 105, 'iexVolume': 153, 'lastTradeTime': 1647988653393, 'latestPrice': 153.72, 'latestSource': 'Close', 'latestTime': 'November 17, 2021', 'latestUpdate': 1643358777239, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2561534797540, 'oddLotDelayedPrice': None, 'oddLotD

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data["latestPrice"]
market_cap = data["marketCap"]
market_cap

2561534797540

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of shares to buy"]
f_DF = pd.DataFrame(columns=my_columns)
f_DF 

,Ticker,Stock Price,Market Capitalization,Number of shares to buy


In [8]:
f_DF.append(
    pd.Series(
        [symbol,
         price,
         market_cap,
         "N/A"
        
        ], index = my_columns
    
    ),
    ignore_index = True

)

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,AAPL,153.72,2561534797540,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [66]:
f_DF = pd.DataFrame(columns=my_columns)
for stock in stocks["Ticker"]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    f_DF = f_DF.append( pd.Series([stock, data["latestPrice"], data["marketCap"], "N/A"], index = my_columns),
        ignore_index = True)

In [67]:
f_DF.size

2020

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks (lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [40]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
f = pd.DataFrame(columns = my_columns) 
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&tocken={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        fin_data = f.append(pd.Series([symbol, data[symbol]["quote"], symbol, "N/A"]), ignore_index = True)
fin_data     

KeyError: 'A'

In [48]:
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&tocken={IEX_CLOUD_API_TOKEN}"
print(batch_api_call_url)


https://sandbox.iexapis.com/stable/stock/market/batch?symbols=YUM,ZBH,ZBRA,ZION,ZTS&types=quote&tocken=Tpk_059b97af715d417d9f49f50b51b1c448


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [71]:
portfolio_size = input("Enter the value of your portfolio: ")
try:
    val = float(portfolio_size)

except ValueError:
    print("Thats not a number! /n Please try again")
    portfolio_size = input("Enter the value of your portfolio: ")
    val = float(portfolio_size)
    
    

Enter the value of your portfolio: 10000000


In [131]:
position_size = val/len(f_DF.index)
for i in range(0, len(f_DF.index)):
    f_DF.loc[i, "Number of shares to buy"] = math.floor(position_size/f_DF.loc[i, "Stock Price"])
f_DF
#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,169.250,49625782415,116
1,AAL,19.812,13023647404,999
2,AAP,233.220,14748866380,84
3,AAPL,164.580,2689690817492,120
4,ABBV,120.170,208256908193,164
...,...,...,...,...
500,YUM,132.580,38187398751,149
501,ZBH,134.330,27710527149,147
502,ZBRA,629.790,32484085338,31
503,ZION,67.490,10819898551,293


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [126]:
writer = pd.ExcelWriter("recommended trades.xlsx", engine = "xlsxwriter")
f_DF.to_excel(writer, "Recommended Trades", index = False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [127]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {"font_color":font_color,
    "bg_color": background_color,
    "border": 1
}
)
dollar_format = writer.book.add_format(
    {"num_format": "$0.00",
    "font_color":font_color,
    "bg_color": background_color,
    "border": 1
}
)
integer_format = writer.book.add_format(
    { "num_format": "0",
        "font_color":font_color,
    "bg_color": background_color,
    "border": 1
}
)



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [136]:
writer.sheets['Recommended Trades'].set_column("A:A", 18, string_format)
writer.sheets['Recommended Trades'].set_column("B:B", 18, string_format)
writer.sheets['Recommended Trades'].set_column("C:C", 18, string_format)
writer.sheets['Recommended Trades'].set_column("D:D", 18, string_format)

writer.sheets['Recommended Trades'].write("A1", "Ticker", string_format)
writer.sheets['Recommended Trades'].write("B1", "Stock Price", dollar_format)
writer.sheets['Recommended Trades'].write("C1", "Market Capitalization", dollar_format)
writer.sheets['Recommended Trades'].write("D1", "Number of shares to buy", integer_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [144]:
column_formats =  {
    "A": ["Ticker", string_format],
    "B": ["Stock Price", dollar_format],
    "C": ["Market Capitalization", dollar_format],
    "D": ["Number of shares to buy", integer_format]
}
for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f"{column}:{column}", 
                                                   18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f"{column}1", 
                                              column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy: